In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = FilterdDataset()
dr = DataReader()

In [4]:
train = dataset.get_train_df()
train_sub = dataset.get_train_df_user_subset()
fd = dr.get_filtered_full_data()

In [21]:
def max_min_time_norm(s):
    min_dat = s.min()
    max_dat = s.max()
    s = ((s - min_dat) / (max_dat - min_dat)) ** 50
    return s

In [22]:
fd.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2


In [32]:
fd["last_buy"] = fd.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
fd["first_buy"] = fd.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
fd["time_score"] = (fd["t_dat"]-fd["first_buy"])/(fd["last_buy"]-fd["first_buy"])

In [33]:
fd.head()

,t_dat,customer_id,article_id,price,sales_channel_id,last,last_buy,first_buy,time_weight
0,2018-09-20,0,0,0.008458,2,2020-04-15,2020-04-15,2018-09-20,0.0
1,2018-09-20,0,0,0.008458,2,2020-04-15,2020-04-15,2018-09-20,0.0
2,2018-09-20,1,1,0.033881,1,2020-08-12,2020-08-12,2018-09-20,0.0
3,2018-09-20,2,2,0.021593,1,2020-03-10,2020-03-10,2018-09-20,0.0
4,2018-09-20,3,3,0.022017,2,2019-09-28,2019-09-28,2018-09-20,0.0


In [30]:
len(fd)

14361088

In [31]:
len(fd[fd["t_dat"] == fd["last"]])

3160399

In [24]:
g = fd.groupby(DEFAULT_USER_COL)["t_dat"]
fd["time_score"] = g.transform(max_min_time_norm)

KeyboardInterrupt: 